In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, UpSampling2D, Concatenate, Cropping2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import random
import shutil
import numpy as np
from tensorflow.keras.layers import Add

# Verify that the file path you are using is correct.
file_path = '/content/drive/MyDrive/RIM-ONE_DL_images/partitioned_randomly/training_set/0001_0001_0001_0001_0001_0001_0001_0001_0001_0001_0000_0001_0000_0001_0000_0000_0000_0000_0000_normal/r3_N-27-L_left_half.png'
print("File path:", file_path)

# Now attempt to access the file
try:
    with open(file_path, "rb") as f:
        # Your file processing code here
        pass
except FileNotFoundError:
    print("File not found. Check the file path.")

# Define your data directories
base_dir_hospital = '/content/drive/MyDrive/RIM-ONE_DL_images/partitioned_by_hospital'
base_dir_random = '/content/drive/MyDrive/RIM-ONE_DL_images/partitioned_randomly'

relative_file_path = '/content/drive/MyDrive/RIM-ONE_DL_images/partitioned_randomly/training_set/0001_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0000_0001_0000_0001_0001_0001_0001_0001_glaucoma/r2_Im420.png'

# Convert to an absolute path
absolute_file_path = os.path.abspath(relative_file_path)

# Verify that the file path you are using is correct.
print("File path:", absolute_file_path)

# Now attempt to access the file
try:
    with open(absolute_file_path, "rb") as f:
        # Your file processing code here
        pass
except FileNotFoundError:
    print("File not found. Check the file path.")

# Shuffle the files in the training directories
def shuffle_directory(directory):
    file_list = os.listdir(directory)
    random.shuffle(file_list)
    for i, filename in enumerate(file_list):
        os.rename(os.path.join(directory, filename), os.path.join(directory, f"{i:04d}_{filename}"))

# Shuffle the training data directories
shuffle_directory(os.path.join(base_dir_hospital, 'training_set'))
shuffle_directory(os.path.join(base_dir_random, 'training_set'))

# Define a custom generator that shuffles the data
def custom_generator(generator1, generator2):
    while True:
        try:
            batch_x1, batch_y1 = next(generator1)
        except StopIteration:
            generator1 = iter(generator1)  # Restart the generator if it's exhausted
            batch_x1, batch_y1 = next(generator1)

        try:
            batch_x2, batch_y2 = next(generator2)
        except StopIteration:
            generator2 = iter(generator2)  # Restart the generator if it's exhausted
            batch_x2, batch_y2 = next(generator2)

        # Shuffle the data within each batch
        indices1 = np.arange(len(batch_x1))
        np.random.shuffle(indices1)
        batch_x1 = batch_x1[indices1]
        batch_y1 = batch_y1[indices1]

        indices2 = np.arange(len(batch_x2))
        np.random.shuffle(indices2)
        batch_x2 = batch_x2[indices2]
        batch_y2 = batch_y2[indices2]

        yield (batch_x1, batch_y1), (batch_x2, batch_y2)

# Image dimensions and batch size.
img_width, img_height = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Create data generators for the "partitioned_by_hospital" folder.
train_generator_hospital = train_datagen.flow_from_directory(
    os.path.join(base_dir_hospital, 'training_set'),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # Change to 'categorical' if you have more than two classes.
)

# Create data generators for the "partitioned_randomly" folder (training set).
train_generator_random = train_datagen.flow_from_directory(
    os.path.join(base_dir_random, 'training_set'),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # Change to 'categorical' if you have more than two classes.
)

# Create data generators for the "partitioned_by_hospital" folder (test set).
test_generator_hospital = test_datagen.flow_from_directory(
    os.path.join(base_dir_hospital, 'test_set'),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # Change to 'categorical' if you have more than two classes.
)


# Create data generators for the "partitioned_randomly" folder (training set).
train_generator_random = train_datagen.flow_from_directory(
    os.path.join(base_dir_random, 'training_set'),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # Change to 'categorical' if you have more than two classes.
)

# Create custom combined generators for testing
test_generator_combined = custom_generator(test_generator_hospital, test_generator_random)

# Combine the generators for both "partitioned_by_hospital" and "partitioned_randomly."
def combined_generator(generator1, generator2):
    while True:
        try:
            batch_x1, batch_y1 = next(generator1)
        except StopIteration:
            generator1 = iter(generator1)  # Restart the generator if it's exhausted
            batch_x1, batch_y1 = next(generator1)

        try:
            batch_x2, batch_y2 = next(generator2)
        except StopIteration:
            generator2 = iter(generator2)  # Restart the generator if it's exhausted
            batch_x2, batch_y2 = next(generator2)

        yield (batch_x1, batch_y1), (batch_x2, batch_y2)

train_generator_combined = combined_generator(train_generator_hospital, train_generator_random)
test_generator_combined = combined_generator(test_generator_hospital, test_generator_random)

# Define the ResU-Net layers (encoder, bottleneck, and decoder).
input_tensor = Input(shape=(img_width, img_height, 3))

# Encoder
encoder_output = Conv2D(64, (3, 3), activation='relu', padding='same')(input_tensor)
encoder_output = MaxPooling2D((2, 2))(encoder_output)
encoder_output = Conv2D(128, (3, 3), activation='relu', padding='same')(encoder_output)
encoder_output = MaxPooling2D((2, 2))(encoder_output)
encoder_output = Conv2D(256, (3, 3), activation='relu', padding='same')(encoder_output)
encoder_output = MaxPooling2D((2, 2))(encoder_output)

# Bottleneck
encoder_output = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder_output)  # Adjusted shape
x = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder_output)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
bottleneck_output = Add()([x, encoder_output])  # Residual connection

# Decoder
x = UpSampling2D((2, 2))(bottleneck_output)
crop_height = x.shape[1]
crop_width = x.shape[2]
encoder_output_adjusted_cropped = Cropping2D(cropping=((0, 0), (0, 0)))(encoder_output)
encoder_output_adjusted_upsampled = UpSampling2D(size=(2, 2))(encoder_output_adjusted_cropped)
x = Concatenate()([x, encoder_output_adjusted_upsampled])  # Skip connection
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
output_tensor = Conv2D(1, (1, 1), activation='sigmoid')(x)

model = Model(inputs=input_tensor, outputs=output_tensor)

# Compile the model.
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)

# Implement early stopping to prevent overfitting.
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

# Calculate the number of steps per epoch for training and validation
steps_per_epoch_train = len(train_generator_hospital) // batch_size
steps_per_epoch_val = len(test_generator_hospital) // batch_size

# Debugging: Print some information about the generators and dataset
print("Train Generator Hospital:", len(train_generator_hospital), "batches,", train_generator_hospital.samples, "samples")
print("Train Generator Random:", len(train_generator_random), "batches,", train_generator_random.samples, "samples")
print("Test Generator Hospital:", len(test_generator_hospital), "batches,", test_generator_hospital.samples, "samples")
print("Test Generator Random:", len(test_generator_random), "batches,", test_generator_random.samples, "samples")

for (batch_x1, batch_y1), (batch_x2, batch_y2) in train_generator_combined:
    print("Batch X1 Shape:", batch_x1.shape)
    print("Batch Y1 Shape:", batch_y1.shape)
    # Add print statements to display file paths or other relevant information.


# Train the model using the custom generators
history = model.fit(
    train_generator_combined,
    steps_per_epoch=steps_per_epoch_train,
    epochs=100,
    validation_data=test_generator_combined,
    validation_steps=steps_per_epoch_val,
    callbacks=[early_stopping]
)

# Evaluate the model on the combined test set.
test_loss, test_accuracy = model.evaluate(test_generator_combined, steps=steps_per_epoch_val)
print(f'Test accuracy: {test_accuracy:.2f}')

# Plot training history.
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


File path: /content/drive/MyDrive/RIM-ONE_DL_images/partitioned_randomly/training_set/0001_0001_0001_0001_0001_0001_0001_0001_0001_0001_0000_0001_0000_0001_0000_0000_0000_0000_0000_normal/r3_N-27-L_left_half.png
File not found. Check the file path.
File path: /content/drive/MyDrive/RIM-ONE_DL_images/partitioned_randomly/training_set/0001_0000_0001_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0000_0001_0000_0001_0000_0001_0001_0001_0001_0001_glaucoma/r2_Im420.png
File not found. Check the file path.
Found 311 images belonging to 2 classes.
Found 339 images belonging to 2 classes.
Found 174 images belonging to 2 classes.
Found 339 images belonging to 2 classes.


NameError: ignored